In [1]:
import cv2, ast
import torchvision
import matplotlib.pyplot as plt
import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
from torchvision import transforms
from torch.autograd import Variable
import torch.functional as F
import pickle
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from PIL import Image, ImageFilter

In [2]:
val_df = pd.read_pickle('lung_cancer_test.pkl')

In [3]:
model = smp.Unet(
    encoder_name="vgg11",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
).cuda()

In [4]:
def dice_coeff(pred, target):
    m1 = np.array(pred).flatten()
    m2 = np.array(target).flatten()
    intersection = np.sum(m1 * m2)
    if (m1.sum() + m2.sum()) == 0: 
        return 0 
    else: 
        return (2. * intersection ) / (m1.sum() + m2.sum() )

In [5]:
from skimage import exposure

In [6]:
np.unique(val_df['mask'][0])

array([0, 1], dtype=int8)

In [7]:
import numpy as np

def calculate_dice_coefficient(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred)
    
    dice_coefficient = (2.0 * intersection) / (union + 1e-8)  # Добавляем маленькое значение для избежания деления на ноль
    
    return dice_coefficient

def precision_score(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean((intersect++ 1e-8)/(total_pixel_pred++ 1e-8))
    return precision

def recall_score(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean((intersect+ 1e-8)/(total_pixel_truth+ 1e-8))
    return recall

def calculate_iou(mask_true, mask_pred):
    intersection = np.sum(mask_true * mask_pred)
    union = np.sum(mask_true) + np.sum(mask_pred) - intersection
    
    iou = (intersection + 1e-8) / (union + 1e-8)  # Добавляем маленькое значение для избежания деления на ноль
    
    return iou
from scipy.spatial.distance import cdist

def hausdorff_distance(mask1, mask2):
    # Получение координат точек в масках
    coords_mask1 = np.transpose(np.nonzero(mask1))
    coords_mask2 = np.transpose(np.nonzero(mask2))

    # Вычисление расстояний между всеми точками в двух масках
    distances_mask1_to_mask2 = cdist(coords_mask1, coords_mask2)
    distances_mask2_to_mask1 = cdist(coords_mask2, coords_mask1)

    # Нахождение максимального расстояния для каждой маски
    max_distance_mask1 = np.max(np.min(distances_mask1_to_mask2, axis=1))
    max_distance_mask2 = np.max(np.min(distances_mask2_to_mask1, axis=1))

    # Вычисление дистанции Хаусдорфа
    hausdorff_distance = max(max_distance_mask1, max_distance_mask2)

    return hausdorff_distance

In [8]:
def model_evaluation(pretrained_weights, method_name):
    model.load_state_dict(torch.load(pretrained_weights))
    model.eval()
    print('augmentation method name: ' + method_name)
    d = []
    iou = []
    prec = []
    rec = []
    labels = val_df['label1'].unique()
    for j in labels:
        dices = []
        test = val_df.loc[val_df['label1'] == j].reset_index(drop=True)
        for i in range(len(test)):
            #new_shape = [224,224]
            mask = test['mask'][i]
            img = test['hu_array'][i]
            img = (img-np.min(img))/(np.max(img)-np.min(img))
            img = exposure.equalize_adapthist(img/np.max(img))
            img = img.astype(np.float64)
            img = transforms.ToTensor()(img)
            img = torch.unsqueeze(img, 0)
            img = img.float()
            img = img.cuda()
            masks = mask.astype(int)
            outputs = model(img)
            outputs = torch.sigmoid(outputs)
            try:
                mask_pred = (outputs.cpu().detach().numpy() >= 0.5) ** 2
            except:
                mask_pred = np.zeros([512,512])
            dices.append(calculate_dice_coefficient(masks, mask_pred))
            d.append(calculate_dice_coefficient(masks, mask_pred))
            iou.append(calculate_iou(masks, mask_pred))
            prec.append(precision_score(masks, mask_pred))
            rec.append(recall_score(masks, mask_pred))
        print("Dice for " + j + " " + str(np.mean(dices))+" " +str(len(dices)))
    print("Average") 
    print("DICE " + str(np.mean(d)))
    print("IoU " + str(np.mean(iou)))
    print("precision " + str(np.mean(prec)))
    print("recall "+str(np.mean(rec)))

In [9]:
model_evaluation('U-Net with VGG/u-net.pth','original images')

augmentation method name: original images
Dice for LR2 0.19973851536424553 35
Dice for LR3 0.3650203799370373 37
Dice for LR4A 0.19791078606168003 97
Dice for LR4B 0.6127724875295943 95
Average
DICE 0.37086111613149014
IoU 0.3120575950341886
precision 0.817595949951946
recall 0.3590172935386154


In [11]:
model_evaluation('U-Net with VGG/u-net_real_mirroflip.pth','mirror flip')

augmentation method name: mirror flip
Dice for LR2 0.059457690248677725 35
Dice for LR3 0.30914355388546594 37
Dice for LR4A 0.34418772956888094 97
Dice for LR4B 0.5938172845342824 95
Average
DICE 0.3913567517098645
IoU 0.32441175523173865
precision 0.8729816500374685
recall 0.3568079688470004


In [13]:
model_evaluation('U-Net with VGG/u-net_real_rotate_image.pth','rotate the image')

augmentation method name: rotate the image
Dice for LR2 0.017810760665590442 35
Dice for LR3 0.3113850298663808 37
Dice for LR4A 0.32398943497859894 97
Dice for LR4B 0.5581194431420177 95
Average
DICE 0.3658823674991194
IoU 0.30751890274276233
precision 0.8697678593680213
recall 0.3196584160067678


In [14]:
model_evaluation('U-Net with VGG/u-net_real_rotate_cancer.pth','Rotate the cancer')

augmentation method name: Rotate the cancer
Dice for LR2 0.02907268169916018 35
Dice for LR3 0.2942263414687766 37
Dice for LR4A 0.08646201909445232 97
Dice for LR4B 0.5426080655237386 95
Average
DICE 0.27211553246489545
IoU 0.2191039259703703
precision 0.887933179607625
recall 0.22500329297878513


In [15]:
model_evaluation('U-Net with VGG/u-net_real_noise_cancer.pth','Noise to image')

augmentation method name: Noise to image
Dice for LR2 0.016254193179330773 35
Dice for LR3 0.31750433289079094 37
Dice for LR4A 0.13144496021883761 97
Dice for LR4B 0.6166499525372229 95
Average
DICE 0.3168502413276488
IoU 0.25962533868550625
precision 0.7967260687902709
recall 0.29120305021594084


In [16]:
model_evaluation('U-Net with VGG/u-net_real_noise_image.pth','Noise to cancer')

augmentation method name: Noise to cancer
Dice for LR2 0.05465637558315465 35
Dice for LR3 0.2993911290357784 37
Dice for LR4A 0.2759910528865982 97
Dice for LR4B 0.5360192098038877 95
Average
DICE 0.3434977347769075
IoU 0.2832515440098847
precision 0.8490878007510665
recall 0.30254330480478825


In [17]:
model_evaluation('U-Net with VGG/u-net_gaussian_bluring.pth','Gaussian blur')

augmentation method name: Gaussian blur
Dice for LR2 0.005370203000091529 35
Dice for LR3 0.11041599588968411 37
Dice for LR4A 0.12750948632251718 97
Dice for LR4B 0.42810012917405277 95
Average
DICE 0.21708818711265418
IoU 0.1600782485429789
precision 0.954241071618539
recall 0.16937242814095313


In [20]:
model_evaluation('U-Net with VGG/u-net_GAN_aug.pth','GAN')

augmentation method name: GAN
Dice for LR2 0.07938721902507102 35
Dice for LR3 0.3176397166939681 37
Dice for LR4A 0.3382312943321709 97
Dice for LR4B 0.613634722487361 95
Average
DICE 0.4001324104927052
IoU 0.3347787145394898
precision 0.7900838486806255
recall 0.37936621538175674


In [19]:
model_evaluation('U-Net with VGG/u-net_VAE_aug.pth','VAE')

augmentation method name: VAE
Dice for LR2 0.016184041180478198 35
Dice for LR3 0.34612719441082934 37
Dice for LR4A 0.3896356018132116 97
Dice for LR4B 0.6261735564257335 95
Average
DICE 0.4191450336016804
IoU 0.3541060241352839
precision 0.7818384845334759
recall 0.38985257734752915
